### Spotify doesn't have it's own API Client. So I have to make one myself.
##### The goal is to make a reusable client id

In [1]:
! pip install requests

In [2]:
import base64
import requests
import datetime
from urllib.parse import urlencode

In [3]:
client_id = "fcbebafbe040467782d5a4ce2a15c4f2"
client_secret = "9ce3301ea98a4bd687364c886baa15de"

In [8]:
class SporifyAPI:
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded string.
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client_id and client_secret")
        
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}" # <base64 encoded client_id:client_secret>
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        token_response_data = r.json()
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data["access_token"]
        expires_in = token_response_data["expires_in"] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type="albums", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def search(self, query, search_type="artist"):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()

In [5]:
client = SporifyAPI(client_id, client_secret)

In [6]:
client.search("watch the throne", search_type="album")

https://api.spotify.com/v1/search?q=watch+the+throne&type=album


{'albums': {'href': 'https://api.spotify.com/v1/search?query=watch+the+throne&type=album&offset=0&limit=20',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
      'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
      'id': '3nFkdlSjzX9mRTtwJOzDYB',
      'name': 'JAY-Z',
      'type': 'artist',
      'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
      'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
      'id': '5K4W6rqBFWDnAN6FQUkS6x',
      'name': 'Kanye West',
      'type': 'artist',
      'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
    'available_markets': ['AD',
     'AE',
     'AL',
     'AR',
     'AT',
     'AU',
     'BA',
     'BE',
     'BG',
     'BH',
     'BO',
     'BR',
     'BY',
     'CA',
     'CH',
     'CL',
     'CO',
     'CR',
 

In [7]:
 client.get_artist("3nFkdlSjzX9mRTtwJOzDYB")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
 'followers': {'href': None, 'total': 5521359},
 'genres': ['east coast hip hop', 'hip hop', 'pop rap', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
 'id': '3nFkdlSjzX9mRTtwJOzDYB',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/4912d27d6b01dd790313d8ef76586be6b100550f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/027c5605d895158497ff377d92d69fe89dba1730',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/1420226e471d0e56ae4de7cb28ba2eb4de0a29ce',
   'width': 160}],
 'name': 'JAY-Z',
 'popularity': 85,
 'type': 'artist',
 'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'}

In [ ]:
client.get_album('')